### 나이브 베이즈 Naive Bayes

#### 베이즈 정리

<img src="https://www.safaribooksonline.com/library/view/doing-data-science/9781449363871/images/dnds_0403.png" align="center" height="600" width="600">

+ $p(sick \ | \ plus)= \ \frac{p(plus \ | \ sick) \ p(sick)}{p(plus)} \ = \ \frac{0.99*0.01}{0.99*0.01+0.01*0.99} \ = \ 0.5 = 50\% \ $

혹은 spam mail 예시

+ 한 단어만 확인
+ $p(spam \ | \ word)= \ \frac{p(word \ | \ spam) \ p(spam)}{p(word)}$
+ $p(word) \ = \ p(word \ |\ spam)p(spam) \ + \ p(word \ | \ ham)p(ham)$  ,,,,,,where  $p(ham)=1-p(spam)$
+ $P(spam \ | \ word) \ = \ [P(word \ | \ spam)P(spam)] \ / \ [P(word \ | \ spam)P(spam) \ + \ P(word \ | \  ham)P(ham)]$

#### 나이브 베이즈

+ 앞의 spam mail 예시를 n 단어로 확장 $w_1, \ w_2, \ ... \ w_n$
+ 나이브 베이즈 : '메시지가 스팸이냐 아니냐가 주어졌다는 조건 하에 각 단어의 존재는 서로 조건부 독립이다.'
+ 나이즈(단순한) 베이즈 : 말이 안되는 가정, 극단적인 가정.
+ 어떤 스팸 메세지에 meeting 이라는 단어를 포함한다는 점이 같은 메시지가 coupon 이라는 단어를 포함하고 있는지 판단하는데 도움을 주지 않는다.  
  
+ $P(X_1=x_1, \ ... \ X_n=x_n \ | \ S) \ = \ P(X_1=x_1 \ | \ S) \times \ ... \ \times P(X_n=x_n \ | \ S)$ 
+ $X_i$ : 단어 $w_i$ 가 메세지에 포함되는 경우의 수 
+ $P(X_i \ | \ S) \ , \ P(X_i \ | \ H)$ : 스팸메일에 단어 $w_i$가 포함될 확률, 스팸메일이 아닌 메일에 포함될 확률  
  
+ 말이 안되는 가정으로 만들어진 모델이지만 성능이 뛰어나다. 

+ $P(S \ | \ X=x) \ = \ P(X=x \ | \ S) \ / \ [P(X=x \ | \ S) \ + \ P(X=x \ | \  ham)]$

#### 구현 & 학습

참고 코드 : https://github.com/Insight-book/data-science-from-scratch/blob/master/notebook/ch13_naive_bayes.ipynb

데이터 다운로드 : http://csmining.org/index.php/spam-assassin-datasets.html

In [1]:
from collections import Counter, defaultdict
import math, random, re, glob

In [2]:
def get_subject_data(path):
    data = []

    # 맨 앞의 "Subject:" 를 지우는 정규식 
    subject_regex = re.compile(r"^Subject:\s+")

    # glob.glob은 주어진 경로에 해당하는 모든 파일 이름 반환
    for fn in glob.glob(path):
        is_spam = "ham" not in fn

        with open(fn,'r',encoding='ISO-8859-1') as file:
            for line in file:
                if line.startswith("Subject:"):
                    subject = subject_regex.sub("", line).strip()
                    data.append((subject, is_spam))
                    # 폴더에서 하나의 파일의 제목을 subject에 그 파일의 스팸 여부를 is_spamn에 저장

    return data

In [19]:
#위에 주제 추출 함수에 파일경로를 입력, 해당 경로의 모든 파일 data에 담기
data = get_subject_data(r"C:/Users/Administrator/Desktop/ML_bayesian/naive_spam/*/*")
data[:5]

#(제목, spam여부) ---현재는 모두 false

[('Re: New Sequences Window', False),
 ('[zzzzteana] RE: Alexander', False),
 ('[zzzzteana] Moscow bomber', False),
 ("[IRR] Klez: The Virus That  Won't Die", False),
 ('Re: Insert signature', False)]

In [16]:
#train, test 비율로 나누기
def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

In [42]:
#train, test 비율 3 : 1 
train_data, test_data = split_data(data, 0.75)
train_data[:5], test_data[:5]

([('[zzzzteana] Moscow bomber', False),
  ("[IRR] Klez: The Virus That  Won't Die", False),
  ('Re: Insert signature', False),
  ('Re: [zzzzteana] Nothing like mama used to make', False),
  ('[zzzzteana] Playboy wants to go out with a bang', False)],
 [('Re: New Sequences Window', False),
  ('[zzzzteana] RE: Alexander', False),
  ('Re: [zzzzteana] Nothing like mama used to make', False),
  ('Re: [zzzzteana] Nothing like mama used to make', False),
  ('[SAtalk] SA CGI Configurator Scripts', False)])

In [43]:
len(train_data), len(test_data)

(2590, 833)

In [20]:
#string을 단어단위로 끊기
def tokenize(message):
    message = message.lower()                       # 소문자 변환
    all_words = re.findall("[a-z0-9']+", message)   # 단어 추출
    return set(all_words)                           # set함수 : 중복제거

In [44]:
tokenize('[zzzzteana] Moscow bomber')

{'bomber', 'moscow', 'zzzzteana'}

In [45]:
tokenize("[IRR] Klez: The Virus That  Won't Die")

{'die', 'irr', 'klez', 'that', 'the', 'virus', "won't"}

In [21]:
#tokenize함수 예시
tokenize("turn the word_counts into a list of triplets")

{'a', 'counts', 'into', 'list', 'of', 'the', 'triplets', 'turn', 'word'}

In [22]:
#단어 횟수 반환
def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

In [46]:
train_data[:5]

[('[zzzzteana] Moscow bomber', False),
 ("[IRR] Klez: The Virus That  Won't Die", False),
 ('Re: Insert signature', False),
 ('Re: [zzzzteana] Nothing like mama used to make', False),
 ('[zzzzteana] Playboy wants to go out with a bang', False)]

In [47]:
# 딕셔너리 .. key : 단어 , [스팸메시지에서 나온 빈도수, 햄메세지에서 나온 빈도수]
count_words(train_data[:5])

defaultdict(<function __main__.count_words.<locals>.<lambda>>,
            {'a': [0, 1],
             'bang': [0, 1],
             'bomber': [0, 1],
             'die': [0, 1],
             'go': [0, 1],
             'insert': [0, 1],
             'irr': [0, 1],
             'klez': [0, 1],
             'like': [0, 1],
             'make': [0, 1],
             'mama': [0, 1],
             'moscow': [0, 1],
             'nothing': [0, 1],
             'out': [0, 1],
             'playboy': [0, 1],
             're': [0, 2],
             'signature': [0, 1],
             'that': [0, 1],
             'the': [0, 1],
             'to': [0, 2],
             'used': [0, 1],
             'virus': [0, 1],
             'wants': [0, 1],
             'with': [0, 1],
             "won't": [0, 1],
             'zzzzteana': [0, 3]})

#### 평활화(smoothing)

+ 만약 주어진 train set에 '데이터'라는 단어는 스팸이 아닌 메세지에만 포함된다라고 가정한다면  
+ $P('data' \ | \ Spam)=0$  
+ 이 경우 나이브 베이즈 분류기는 '데이터'라는 단어가 들어간 메시지를 항상 스팸메시지가 아니라고 예측  
+ '만남 주선 쿠폰에 대한 데이터' 라는 메시지를 스팸이 아니라고 예측하게 됨. 
+ 이러한 문제를 해결하기 위해 일종의 평활화가 필요

+ smoothing을 위해 가짜 빈도수 (psedocount) k 를 결정하고 스팸 메세지에서 i 번째 단어가 나올 확률을 아래와 같이 추정

+ $P(X_i \ | \ Spam) \ = \ (k \ + \ number \ of \ spam \ massage \ including \ w_i) \ / \ (2k \ + \ number \ of \  spam \ massage)$

+ 예를 들어 '데이터'라는 단어는 98개의 스팸문서에서 단 한번도 나타나지 않았음
+ 그러나 k가 1이라면 P('데이터|S)를 1/100 = 0.01 로 계산할 수 있음
+ 즉, '데이터'라는 단어가 포함된 메세지가 스팸메세지일 확률을 0이 아닌 다른 확률값으로 설정할 수 있게 해준다.

In [24]:
#평활화
#단어의 빈도수를 [단어,p(w|spam),p(w|ham)]의 형태로 변환
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets
    w, p(w | spam) and p(w | ~spam)"""
    return [(w,
             (spam + k) / (total_spams + 2 * k),
             (non_spam + k) / (total_non_spams + 2 * k))
             for w, (spam, non_spam) in counts.items()]

In [25]:
#평활화 결과
word_probabilities(
    count_words(train_data), len(
        [is_spam for message, is_spam in train_data if is_spam]), 
    1-len([is_spam for message, is_spam in train_data if is_spam]))[:5]

[('outstanding', 0.004087193460490463, -0.0013736263736263737),
 ('pl', 0.0013623978201634877, -0.006868131868131868),
 ('slouch', 0.0013623978201634877, -0.004120879120879121),
 ('nr', 0.004087193460490463, -0.0013736263736263737),
 ('fire', 0.0013623978201634877, -0.009615384615384616)]

In [26]:
def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0

    for word, prob_if_spam, prob_if_not_spam in word_probs:

        # for each word in the message,
        # add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)

        # for each word that's not in the message
        # add the log probability of _not_ seeing it
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [50]:
def p_spam_given_word(word_prob):
    #베이즈 정리 : p(spam|메세지가 해당 단어를 포함)을 계산
    #word_prob은 word_probabilities에서 계산된 결과
    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [28]:
#나이브 베이즈 분류기 클래스 생성
class NaiveBayesClassifier:
    #생성자 --> 초기화
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def train(self, training_set):
        #스팸 메시지와 스팸이 아닌 메시지의 개수 세기
        # count spam and non-spam messages
        num_spams = len([is_spam
                         for message, is_spam in training_set
                         if is_spam])
        num_non_spams = len(training_set) - num_spams
        
        #지금까지 만든 함수에 train set 적용
        # run training data through our "pipeline"
        #단어 횟수 반환
        word_counts = count_words(training_set)
        #평활화 함수
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k)

    def classify(self, message):
        return spam_probability(self.word_probs, message)

In [29]:
def train_and_test_model(path):

    data = get_subject_data(path)
    random.seed(0)      # just so you get the same answers as me
    train_data, test_data = split_data(data, 0.75)

    classifier = NaiveBayesClassifier()
    classifier.train(train_data)
    
    #[제목, 실제 스팸 여부, 예측 스팸 확률]
    classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]
    
    #메세지가 스팸일 확률이 0.5 보다 크면 스팸!
    counts = Counter((is_spam, spam_probability > 0.5) # (actual, predicted)
                     for _, is_spam, spam_probability in classified)

    print(counts)
    
    #스팸일 확률을 오름차순 정렬
    classified.sort(key=lambda row: row[2])
    
    #스팸이 아닌 메시지 중에서 스팸일 확률이 가장 높은 메시지
    spammiest_hams = list(filter(lambda row: not row[1], classified))[-5:]
    
    #스팸 중에서 스팸일 확률이 가장 낮은 메시지
    hammiest_spams = list(filter(lambda row: row[1], classified))[:5]

    print("spammiest_hams", spammiest_hams)
    print("hammiest_spams", hammiest_spams)

    words = sorted(classifier.word_probs, key=p_spam_given_word)

    spammiest_words = words[-5:]
    hammiest_words = words[:5]

    print("spammiest_words", spammiest_words)
    print("hammiest_words", hammiest_words)

In [30]:
#실행
if __name__ == "__main__":
    #train_and_test_model(r"c:\spam\*\*")
    train_and_test_model(r"C:/Users/Administrator/Desktop/ML_bayesian/naive_spam/*/*")

Counter({(False, False): 704, (True, True): 101, (True, False): 38, (False, True): 33})
spammiest_hams [('Attn programmers: support offered [FLOSS-Sarai Initiative]', False, 0.975612960514211), ('2000+ year old Greek computer reinterpreted', False, 0.9835355008104786), ('What to look for in your next smart phone (Tech Update)', False, 0.9898719206903309), ('[ILUG-Social] Re: Important - reenactor insurance needed', False, 0.9995349057803384), ('[ILUG-Social] Re: Important - reenactor insurance needed', False, 0.9995349057803384)]
hammiest_spams [('Re: girls', True, 0.0009525186158414364), ('Introducing Chase Platinum for Students with a 0% Introductory APR', True, 0.0012566691211088627), ('.Message report from your contact page....//ytu855 rkq', True, 0.0015109358288608922), ('Testing a system, please delete', True, 0.0026920538836805305), ('Never pay for the goodz again (8SimUgQ)', True, 0.005911623221926722)]
spammiest_words [('year', 0.028767123287671233, 0.00022893772893772894), ('